<a href="https://colab.research.google.com/github/carsofferrei/04_data_processing/blob/main/spark_streaming/challenges/final_challenges.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting up PySpark

In [1]:
%pip install pyspark

# Context
Message events are coming from platform message broker (kafka, pubsub, kinesis...).
You need to process the data according to the requirements.

Message schema:
- timestamp
- value
- event_type
- message_id
- country_id
- user_id



# Challenge 1

Step 1
- Change exising producer
	- Change parquet location to "/content/lake/bronze/messages/data"
	- Add checkpoint (/content/lake/bronze/messages/checkpoint)
	- Delete /content/lake/bronze/messages and reprocess data
	- For reprocessing, run the streaming for at least 1 minute, then stop it

Step 2
- Implement new stream job to read from messages in bronze layer and split result in two locations
	- "messages_corrupted"
		- logic: event_status is null, empty or equal to "NONE"
    - extra logic: add country name by joining message with countries dataset
		- partition by "date" -extract it from timestamp
		- location: /content/lake/silver/messages_corrupted/data

	- "messages"
		- logic: not corrupted data
		- extra logic: add country name by joining message with countries dataset
		- partition by "date" -extract it from timestamp
		- location: /content/lake/silver/messages/data

	- technical requirements
		- add checkpint (choose location)
		- use StructSchema
		- Set trigger interval to 5 seconds
		- run streaming for at least 20 seconds, then stop it

	- alternatives
		- implementing single streaming job with foreach/- foreachBatch logic to write into two locations
		- implementing two streaming jobs, one for messages and another for messages_corrupted
		- (paying attention on the paths and checkpoints)


  - Check results:
    - results from messages in bronze layer should match with the sum of messages+messages_corrupted in the silver layer

In [2]:
%pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 19.1 MB/s eta 0:00:00


In [ ]:
!mkdir -p /content/lake/bronze
!mkdir -p /content/lake/silver
!mkdir -p /content/lake/gold

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory


# Producer

In [3]:
import pyspark.sql.functions as F
from pyspark.sql import DataFrame
from faker import Faker
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('Test streaming').getOrCreate()
sc = spark.sparkContext

fake = Faker()
messages = [fake.uuid4() for _ in range(50)]

def enrich_data(df, messages=messages):
  fake = Faker()
  new_columns = {
      'event_type': F.lit(fake.random_element(elements=('OPEN', 'RECEIVED', 'SENT', 'CREATED', 'CLICKED', '', 'NONE'))),
      'message_id': F.lit(fake.random_element(elements=messages)),
      'channel': F.lit(fake.random_element(elements=('CHAT', 'EMAIL', 'SMS', 'PUSH', 'OTHER'))),
      'country_id': F.lit(fake.random_int(min=2000, max=2015)),
      'user_id': F.lit(fake.random_int(min=1000, max=1050)),
  }
  df = df.withColumns(new_columns)
  return df

def insert_messages(df: DataFrame, batch_id):
  enrich = enrich_data(df)
  enrich.write.mode("append").format("parquet").save("/content/lake/bronze/messages/data")

# read stream
df_stream = spark.readStream.format("rate").option("rowsPerSecond", 1).load()

# write stream
query = (df_stream
         .writeStream
         .outputMode('append')
         .option('checkpointLocation', "/content/lake/bronze/checkpoint")
         .trigger(processingTime='1 seconds')
         .foreachBatch(insert_messages)
         .start()
)

query.awaitTermination(60)


False

In [4]:
query.isActive

True

In [5]:
query.stop()

In [7]:
df = spark.read.format("parquet").load("lake/bronze/messages/data/*")
df.show()

+--------------------+-----+----------+--------------------+-------+----------+-------+
|           timestamp|value|event_type|          message_id|channel|country_id|user_id|
+--------------------+-----+----------+--------------------+-------+----------+-------+
|2024-12-10 21:38:...|    0|      NONE|3aeb6332-394f-4f9...|    SMS|      2015|   1029|
|2024-12-10 21:38:...|    2|      NONE|3aeb6332-394f-4f9...|    SMS|      2015|   1029|
|2024-12-10 21:38:...|    1|      NONE|3aeb6332-394f-4f9...|    SMS|      2015|   1029|
|2024-12-10 21:38:...|    3|      NONE|3aeb6332-394f-4f9...|    SMS|      2015|   1029|
|2024-12-10 21:39:...|   70|  RECEIVED|5a404e2d-0a37-4a3...|  EMAIL|      2010|   1013|
|2024-12-10 21:38:...|   32|  RECEIVED|15aa6dbe-8ddd-468...|  EMAIL|      2006|   1017|
|2024-12-10 21:39:...|   69|  RECEIVED|7efcb726-4d35-4f8...|  OTHER|      2004|   1042|
|2024-12-10 21:38:...|    4|  RECEIVED|810166bd-4fc9-485...|  OTHER|      2000|   1013|
|2024-12-10 21:38:...|   19|  RE

# Additional datasets

In [8]:
countries = [
    {"country_id": 2000, "country": "Brazil"},
    {"country_id": 2001, "country": "Portugal"},
    {"country_id": 2002, "country": "Spain"},
    {"country_id": 2003, "country": "Germany"},
    {"country_id": 2004, "country": "France"},
    {"country_id": 2005, "country": "Italy"},
    {"country_id": 2006, "country": "United Kingdom"},
    {"country_id": 2007, "country": "United States"},
    {"country_id": 2008, "country": "Canada"},
    {"country_id": 2009, "country": "Australia"},
    {"country_id": 2010, "country": "Japan"},
    {"country_id": 2011, "country": "China"},
    {"country_id": 2012, "country": "India"},
    {"country_id": 2013, "country": "South Korea"},
    {"country_id": 2014, "country": "Russia"},
    {"country_id": 2015, "country": "Argentina"}
]

countries = spark.createDataFrame(countries)

# Streaming Messages x Messages Corrupted

In [16]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

def cleansing_silver(df: DataFrame):
  messages = spark.read.format("parquet").load("lake/bronze/messages/data/*")
  print(f"DataFrame from bronze have {messages.count()} rows")

  print(f"Joining country information to dataframe")
  messages = messages.join(countries, "country_id", "left")
  messages = messages.withColumn("date", col('timestamp').cast("date"))

  print(f'Filtering corrupted messages')
  corrupted_df = messages.filter((col("event_type").isNull() |
                              (col("event_type") == "") |
                              (col("event_type") == "NONE")
                              ))
  print(f'Saving corrupted messages on: /content/lake/silver/messages_corrupted/data')
  corrupted_df.write.mode("append").format("parquet").partitionBy("date").save("/content/lake/silver/messages_corrupted/data")
  print(f"Corrupted DataFrame have {corrupted_df.count()} rows")

  print(f'Creating dataframe without messages')
  clean_df = messages.filter(~(col("event_type").isNull() |
                          (col("event_type") == "") |
                          (col("event_type") == "NONE"))
                        )
  print(f'Saving corrupted messages on: /content/lake/silver/messages/data')
  clean_df.write.mode("append").format("parquet").partitionBy("date").save("/content/lake/silver/messages/data")
  print(f"Clean DataFrame have {clean_df.count()} rows")

DataFrame from bronze have 85 rows
Filtering corrupted messages
Saving corrupted messages on: /content/lake/silver/messages_corrupted/data
Creating dataframe without messages
Saving corrupted messages on: /content/lake/silver/messages/data
Corrupted DataFrame have 26 rows
Clean DataFrame have 59 rows


In [17]:
clean_schema = StructType([StructField('timestamp', TimestampType(), True),
                     StructField('date', DateType(), True),
                     StructField('user_id', IntegerType(), True),
                     StructField('value', IntegerType(), True),
                     StructField('event_type', StringType(), True),
                     StructField('channel', StringType(), True),
                     StructField('message_id', StringType(), True),
                     StructField('country_id', IntegerType(), True),
                     StructField('country', StringType(), True),
                    ])

# Read the streaming data
messages = (spark.readStream
               .format("parquet")
               .schema(clean_schema)
               .load("content/lake/bronze/messages/data/*")
)

# Write the streaming data with partitioning
query = (messages.writeStream
         .format("parquet")
         .outputMode("append")
         .option("checkpointLocation", "/content/lake/silver/messages/_checkpoints/")
         .trigger(processingTime='1 seconds')
         .foreachBatch(cleansing_silver)
         .start()
)


In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
import time

def cleansing_silver(df: DataFrame):
    if df.isEmpty():
        print("No data in the current batch.")
        return

    print(f"Processing batch with {df.count()} rows.")

    # Join with countries
    df = df.join(broadcast(countries), "country_id", "left")
    df = df.withColumn("date", col('timestamp').cast("date"))

    print(f'Filtering corrupted messages...')
    corrupted_df = messages.filter((col("event_type").isNull() |
                              (col("event_type") == "") |
                              (col("event_type") == "NONE")
                              ))

    try:
        print(f'Saving corrupted messages...')
        corrupted_df.write.mode("append").format("parquet").partitionBy("date").save("/content/lake/silver/messages_corrupted/data")
        print(f"Corrupted DataFrame has {corrupted_df.count()} rows.")
    except Exception as e:
        print(f"Error saving corrupted messages: {e}")

    print(f'Creating dataframe without corrupted messages...')
    clean_df = messages.filter(~(col("event_type").isNull() |
                          (col("event_type") == "") |
                          (col("event_type") == "NONE"))
                        )

    try:
        print(f'Saving clean messages...')
        clean_df.write.mode("append").format("parquet").partitionBy("date").save("/content/lake/silver/messages/data")
        print(f"Clean DataFrame has {clean_df.count()} rows.")
    except Exception as e:
        print(f"Error saving clean messages: {e}")


print(f'Define streaming schema...')
clean_schema = StructType([
    StructField('timestamp', TimestampType(), True),
    StructField('date', DateType(), True),
    StructField('user_id', IntegerType(), True),
    StructField('value', IntegerType(), True),
    StructField('event_type', StringType(), True),
    StructField('channel', StringType(), True),
    StructField('message_id', StringType(), True),
    StructField('country_id', IntegerType(), True),
    StructField('country', StringType(), True),
])

print(f'Read the streaming data...')
messages = (spark.readStream
               .format("parquet")
               .schema(clean_schema)
               .load("content/lake/bronze/messages/data/*")
)

print(f'Write the streaming data and guarantee a checkpoint...')
query = (messages.writeStream
         .format("parquet")
         .outputMode("append")
         .option("checkpointLocation", "/content/lake/silver/messages/_checkpoints/")
         .trigger(processingTime='5 seconds')
         .foreachBatch(cleansing_silver)
         .start()
)

time.sleep(20)
query.stop()

In [ ]:
def validate_results():
    bronze_count = spark.read.format("parquet").load("content/lake/bronze/messages/data/*").count()
    clean_count = spark.read.format("parquet").load("/content/lake/silver/messages/data").count()
    corrupted_count = spark.read.format("parquet").load("/content/lake/silver/messages_corrupted/data").count()

    assert bronze_count == clean_count + corrupted_count, "Counts do not match!"
    print("Validation passed: Bronze = Clean + Corrupted")

In [ ]:
# TODO

False

In [ ]:
df = spark.read.format("parquet").load("lake/silver/messages/data/*")
df.show()

+--------------------+----------+
|           timestamp|event_type|
+--------------------+----------+
|2024-12-09 22:34:...|  RECEIVED|
|2024-12-09 22:34:...|  RECEIVED|
|2024-12-09 22:33:...|  RECEIVED|
|2024-12-09 22:34:...|  RECEIVED|
|2024-12-09 22:34:...|  RECEIVED|
|2024-12-09 22:34:...|  RECEIVED|
|2024-12-09 22:34:...|  RECEIVED|
|2024-12-09 22:34:...|   CLICKED|
|2024-12-09 22:34:...|   CREATED|
|2024-12-09 22:33:...|   CLICKED|
|2024-12-09 22:34:...|  RECEIVED|
|2024-12-09 22:34:...|   CREATED|
|2024-12-09 22:34:...|   CLICKED|
|2024-12-09 22:34:...|   CLICKED|
|2024-12-09 22:34:...|   CREATED|
|2024-12-09 22:33:...|   CREATED|
|2024-12-09 22:33:...|   CLICKED|
|2024-12-09 22:33:...|   CLICKED|
|2024-12-09 22:34:...|   CREATED|
|2024-12-09 22:33:...|  RECEIVED|
+--------------------+----------+
only showing top 20 rows



## Checking data

In [ ]:
# TODO

# Challenge 2

- Run business report
- But first, there is a bug in the system which is causing some duplicated messages, we need to exclude these lines from the report

- removing duplicates logic:
  - Identify possible duplicates on message_id, event_type and channel
  - in case of duplicates, consider only the first message (occurrence by timestamp)
  - Ex:
    In table below, the correct message to consider is the second line

```
    message_id | channel | event_type | timestamp
    123        | CHAT    | CREATED    | 10:10:01
    123        | CHAT    | CREATED    | 07:56:45 (first occurrence)
    123        | CHAT    | CREATED    | 08:13:33
```

- After cleaning the data we're able to create the busines report

In [ ]:
# dedup data
from pyspark.sql import functions as F
from pyspark.sql.window import Window
df = spark.read.format("parquet").load("content/lake/silver/messages")
dedup = df.withColumn("row_number", F.row_number().over(Window.partitionBy("message_id", "event_type", "channel").orderBy("timestamp"))).filter("row_number = 1").drop("row_number")

### Report 1
  - Aggregate data by date, event_type and channel
  - Count number of messages
  - pivot event_type from rows into columns
  - schema expected:
  
```
|      date|channel|CLICKED|CREATED|OPEN|RECEIVED|SENT|
+----------+-------+-------+-------+----+--------+----+
|2024-12-03|    SMS|      4|      4|   1|       1|   5|
|2024-12-03|   CHAT|      3|      7|   5|       8|   4|
|2024-12-03|   PUSH|   NULL|      3|   4|       3|   4|
```

In [ ]:
# report 1
# TODO

## Report 2

- Identify the most active users by channel (sorted by number of iterations)
- schema expected:

```
+-------+----------+----+-----+-----+----+---+
|user_id|iterations|CHAT|EMAIL|OTHER|PUSH|SMS|
+-------+----------+----+-----+-----+----+---+
|   1022|         5|   2|    0|    1|   0|  2|
|   1004|         4|   1|    1|    1|   1|  0|
|   1013|         4|   0|    0|    2|   1|  1|
|   1020|         4|   2|    0|    1|   1|  0|
```


In [ ]:
# report 2
# TODO

# Challenge 3

In [ ]:
# Theoretical question:

# A new usecase requires the message data to be aggregate in near real time
# They want to build a dashboard embedded in the platform website to analyze message data in low latency (few minutes)
# This application will access directly the data aggregated by streaming process

# Q1:
- What would be your suggestion to achieve that using Spark Structure Streaming?
Or would you choose a different data processing tool?

- Which storage would you use and why? (database?, data lake?, kafka?)

